In [ ]:
import keras
from keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import cv2
import os

# Load the dataset
def load_dataset(data_directory):
    images = []
    labels = []
    
    # Iterate over the subdirectories (classes) in the data directory
    for class_dir in os.listdir(data_directory):
        class_path = os.path.join(data_directory, class_dir)
        if os.path.isdir(class_path):
            # Iterate over the images in the class directory
            for img_filename in os.listdir(class_path):
                img_path = os.path.join(class_path, img_filename)
                # Load the image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                # Resize the image to a consistent size (e.g., 128x128)
                img_resized = cv2.resize(img, (512, 512))
                # Normalize the image
                img_normalized = img_resized.astype('float32') / 255.0
                # Append the image and label to the lists
                images.append(img_normalized)
                labels.append(class_dir)
    
    return np.array(images), np.array(labels)

# Define the ANN architecture for binary classification
def create_ann_model(input_shape):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(512, activation='relu'),  # First hidden layer with 512 units and ReLU activation
        layers.Dense(128, activation='relu'),  # Second hidden layer with 128 units and ReLU activation
        layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
    ])
    
    return model

# Load the dataset
data_directory = 'medical_images'  # Specify the directory containing the medical images dataset
images, labels = load_dataset(data_directory)

# Convert string labels to binary format (0 or 1)
label_mapping = {label: idx for idx, label in enumerate(np.unique(labels))}
y_binary = np.array([label_mapping[label] for label in labels])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, y_binary, test_size=0.2, random_state=42)

# Define the ANN model
input_shape = (512, 512)
model = create_ann_model(input_shape)

# Compile the model
model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



In [ ]:

keras.models.save_model(model, filepath='ann_medical_model.keras', overwrite=True)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Print classification report
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

# Plot training history
plt.figure()
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Load the trained model
model = keras.models.load_model('ann_medical_model.keras')

# Function to preprocess a new medical image
def preprocess_image(image_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image to the size expected by the model (e.g., 128x128)
    img_resized = cv2.resize(img, (512, 512))
    # Normalize the image
    img_normalized = img_resized.astype('float32') / 255.0
    # Add batch dimension (e.g., 1, 128, 128)
    img_normalized = np.expand_dims(img_normalized, axis=0)
    return img_normalized

# Path to the new medical image
new_image_path = 'medical_images/Brain Tumor/Cancer (2399).jpg'

# Preprocess the new image
new_image = preprocess_image(new_image_path)

# Make a prediction using the trained model
prediction = model.predict(new_image)
print(prediction)
plt.imshow(new_image[0], cmap='gray')
# Interpret the prediction
# If the model is binary classification, consider a threshold to determine the result
threshold = 0.5
if prediction >= threshold:
    print(f"Healthy Brain (probability: {prediction[0][0]*100:.4f}%)")
else:
    print(f"Brain Tumour (probability: {100-100*prediction[0][0]:.4f}%)")


In [ ]:

# Path to the new medical image
new_image_path = 'medical_images/Healthy Brain/Not Cancer  (13).jpg'

# Preprocess the new image
new_image = preprocess_image(new_image_path)

# Make a prediction using the trained model
prediction = model.predict(new_image)
print(prediction)
plt.imshow(new_image[0], cmap='gray')
# Interpret the prediction
# If the model is binary classification, consider a threshold to determine the result
threshold = 0.5
if prediction >= threshold:
    print(f"Healthy Brain (probability: {prediction[0][0]*100:.4f}%)")
else:
    print(f"Brain Tumour (probability: {100-100*prediction[0][0]:.4f}%)")
